# Using Xpress in python

Before we can start using the Xpress package that we have just installed we will need to begin by importing it.

In [ ]:
import xpress as xp
import pandas as pd
import numpy as np

Lets begin by writing some code to solve the simple linear programming problem

min $z = x +y$

s.t. 

$2x + y \ge 1$,

$x +2y \ge 1$,
     
$ -1 \ge y \le 1$.

The first thing we need to do is create a problem. 

In [ ]:
# Create a problem called myproblem

prob = xp.problem(name='myproblem')

We are now ready to define our variables using the xp.var function:

`var(name, lb, ub, threshold, vartype)`
 
The `name`, `lb` and `up` arguments are self explanatory. The `threshold` argument is the threshold for semi-continuous, semi-integer, and partially integer variables and the `vartype` argument is used to set the variable type (`continuous`, `binary`, `integer` ect.).

Once the variables are defined we will add them to our problem.

In [ ]:
# Create the x and y variables
x = xp.var('x')
y = xp.var('y',lb=-1, ub=1)

# Add the variables to the problem

prob.addVariable(x,y)

We can now begin to add our constraints. To do this we will use the xp.constraints function:

In [ ]:
prob.addConstraint(2*x + y >= 1)
prob.addConstraint(x + 2*y >= 1)

We can now define our objective function using the `xp.setObjective()` function:

`setObjective(objective, sense)`

The `objective` is the expression defining the new objective and `sense` is either `xp.minimize` or `xp.maximize`(note that it must be American spelling).

In [ ]:
obj = x + y
prob.setObjective(obj, sense = xp.minimize)

We can now solve the problem!

In [ ]:
xp.setOutputEnabled(True)
prob.solve()

In the above cell, we can turn off the log output by changing `xp.setOutputEnabled(True)` to `False`. 

It might now be a good idea to print out the results.

In [ ]:
print(f'x =  {prob.getSolution(x)}')
print(f'y =  {prob.getSolution(y)}') 
print(f'The objective function value is {prob.getObjVal()}') 

This is a very simple example, lets try to solve a more complex problem. Remember the sailco example from the week 2 lecture.

$$\begin{array}{lll}
		 \min & \displaystyle\sum_{t=1}^4 400x_{1t} + \sum_{t=1}^4 35x_{2t} + \sum_{t=1}^4 50i_{1t} + \sum_{t=1}^4 2i_{2t}\\
		 \text{s.t.} & 20x_{1t} + 3x_{2t} \leq 1860, & t=1,2,3,4,\\
		 & y_{it}\geq d_{i,t}, &  t=1,2,3,4,\\
		 & i_{p1} = x_{p1} - y_{p1}, & p=1,2,\\
		 & i_{pt} = i_{p,t-1} + x_{pt} - y_{pt}, & p=1,2, \ t=2,3,4,\\
		 & i_{pt}, x_{pt}, y_{pt} \geq 0, & p=1,2, \ t=1,2,3,4
		 \end{array}$$
with demand 


|           | Spring | Summer | Autumn | Winter |
|-----------|--------|--------|--------|--------|
| Sailboat  | 40     | 60     | 75     | 25     |
| Surfboard | 190    | 350    | 130    | 20     |

lets solve this problem. Lets begin by creating a numpy array containing our demand data.

In [ ]:
# Creating the demand matrix

demand = np.array([[ 40, 60, 75, 25],[190, 350, 130, 20]])
print(demand)

# Creating lists with seasons and products

product_names = ['Sailboat', 'Surfboard']
season_names = ['Spring', 'Summer', 'Autumn', 'Winter']

Now lets define our index sets.

In [ ]:
# Defining the index sets

number_of_products = 2
number_of_periods = 4
Products = range(number_of_products)
Periods = range(number_of_periods)

Now lets define our problem and decision variables using numpy arrays.

In [ ]:
# Define the problem
prob = xp.problem('Sailco')

# Define our decision variable as a numpy array
# We use the reshape function to make the numpy array a matrix
make = np.array([xp.var( name='make_{0}_{1}'.format(i+1,j+1))
                    for i in Products for j in Periods], dtype=xp.npvar).reshape(number_of_products,number_of_periods)
sell = np.array([xp.var( name='sell_{0}_{1}'.format(i+1,j+1))
                    for i in Products for j in Periods], dtype=xp.npvar).reshape(number_of_products,number_of_periods)
inventory = np.array([xp.var( name='inventory_{0}_{1}'.format(i+1,j+1))
                    for i in Products for j in Periods], dtype=xp.npvar).reshape(number_of_products,number_of_periods)

print(make)

prob.addVariable(make,sell,inventory)

Lets add our constraints

In [ ]:
# Labour restriction
prob.addConstraint(20*make[0,t] + 3*make[1,t] <= 1860 for t in Periods)

# Demand constraints
prob.addConstraint(sell[i,t] >= demand[i,t] for i in Products for t in Periods)

# Inventory ballance for all times except first time period
prob.addConstraint(inventory[i,t] == inventory[i,t-1] + make[i,t] - sell[i,t] for i in Products for t in Periods if t != 0)


# Inventory ballance for first time period
prob.addConstraint(inventory[i,0] == make[i,0] -sell[i,0] for i in Products)

Now lets add our objective function. Note how we use the the sum function contained in the Xpress module rather than the native python operator. This is for efficiency reasons 

In [ ]:
# Define the objective function

prob.setObjective(400*xp.Sum(make[0,t] for t in Periods) + 35*xp.Sum(make[1,t] for t in Periods) + 
                  50*xp.Sum(inventory[0,t] for t in Periods) +  2*xp.Sum(inventory[1,t] for t in Periods), 
                  sense = xp.minimize)

Sometimes it is good to check our problem before we solve it, we can do this by saving the corresponding `lp` file. This is very useful for debugging.

In [ ]:
prob.write("problem","lp")

In [ ]:
xp.setOutputEnabled(False)
prob.solve()

Lets add our solutions to data frames. 

In [ ]:
# Add the solutions into a dataframe

make_df = pd.DataFrame(data = prob.getSolution(make), index = product_names, columns = season_names)
sell_df = pd.DataFrame(data = prob.getSolution(sell), index = product_names, columns = season_names)
inventory_df = pd.DataFrame(data = prob.getSolution(inventory), index = product_names, columns = season_names)

make_df = make_df.style.set_caption('Make')
sell_df = sell_df.style.set_caption('Sell')
inventory_df = inventory_df.style.set_caption('Inventory')

display(make_df)
display(sell_df)
display(inventory_df)

In some cases we might want to use dictionaries to define our decision variables. Lets consider the sudoku problem from week. We will start off by defining the dimension of the sudoku puzzle and by creating the initial puzzle.

In [ ]:
# The input is a starting grid where the unknown numbers are replaced by zero
# q is the dimension of the sub blocks
q = 3

starting_grid = \
 [[8, 0, 0, 0, 0, 0, 0, 0, 0],
  [0, 0, 3, 6, 0, 0, 0, 0, 0],
  [0, 7, 0, 0, 9, 0, 2, 0, 0],
  [0, 5, 0, 0, 0, 7, 0, 0, 0],
  [0, 0, 0, 0, 4, 5, 7, 0, 0],
  [0, 0, 0, 1, 0, 0, 0, 3, 0],
  [0, 0, 1, 0, 0, 0, 0, 6, 8],
  [0, 0, 8, 5, 0, 0, 0, 1, 0],
  [0, 9, 0, 0, 0, 0, 4, 0, 0]]

We will now define our decision variables $x_{i,j,k}$ as a dictionary.

In [ ]:
n = q**2  # the size must be the square of the size of the subgrids
N = range(n)

x = {(i, j, k): xp.var(vartype=xp.binary, name='x{0}_{1}_{2}'.format(i, j, k))
     for i in N for j in N for k in N}

print(x)

Before we define our constraints lest define the box index set.

In [ ]:
# define all q^2 subgrids
subgrids = {(h, l): [(i, j) for i in range(q*h, q*h + q)
            for j in range(q*l, q*l + q)]
            for h in range(q)
            for l in range(q)}
print(subgrids)

Now lets define all the constraints

In [ ]:
vertical = [xp.Sum(x[i, j, k] for i in N) == 1 for j in N for k in N]
horizontal = [xp.Sum(x[i, j, k] for j in N) == 1 for i in N for k in N]
subgrid = [xp.Sum(x[i, j, k] for (i, j) in subgrids[h, l]) == 1
           for (h, l) in subgrids.keys() for k in N]

# Assign exactly one number to each cell

assign = [xp.Sum(x[i, j, k] for k in N) == 1 for i in N for j in N]

# Fix those variables that are non-zero in the input grid

init = [x[i, j, k] == 1 for k in N for i in N for j in N
        if starting_grid[i][j] == k+1]
print(vertical)

We are now ready to define the problem, add our constraints and decision variables.

In [ ]:
# Define the problem
p = xp.problem()

# Include decision variables
p.addVariable(x)

# Include constraints
p.addConstraint(vertical, horizontal, subgrid, assign, init)

Now lets solve the problem!

In [ ]:
p.solve()

print('Solution:')

for i in N:
    for j in N:
        l = [k for k in N if p.getSolution(x[i, j, k]) >= 0.5]
        assert(len(l) == 1)
        print('{0:2d}'.format(1 + l[0]), end='', sep='')
    print('')